<a href="https://colab.research.google.com/github/HwangHanJae/NLP_tutorial/blob/main/Dacon_NLP_tutorial_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Embedding 이란?

vectorization의 치명적인 단점은 바로 단어나 문장들 사이의 관계에 대해서 설명하지 못하는 것입니다.
  - ont hot encoding을 통해 각 5개의 토큰들이 고유의 벡터를 갖게 되었습니다. 사람이 보기에는 5개의 토큰들이 너무나도 명확하게 구분이되지만 단어의 중요도나 문서안에서의 중요도는 구분할 수 있지만 단어 사이의 유사도는 구분할 수 없습니다.
  
이때 사용되는 것이 Embedding 기법이며 word2vec을 비롯한 다양한 임베딩 기법들이 존재합니다.

비슷한 의미를 내포하고 있는 토큰들은 서로 가깝게, 그렇지 않은 토큰들은 서로 멀리 뿌리도록 하는 것이 embedding의 목적입니다.

검색 시스템, 감성 분석 등에서는 훌륭한 임베딩을 수행하는 것이 전체 문제 해결에 많은 영향을 줍니다.

임베딩 또한 하나의 모델을 의미하며 훈련이 필요합니다. 데이터가 충분하고 시간이 많으면 소지한 데이터에 특화된 임베딩 모델을 학습 시킬 수 있습니다.

보통은 pre_trained embedding model을 가져와서 사용합니다.

# 1. keras Embedding Layer
- 기본적으로 가장 쉽고 빠르게 네트워크 모델에 임베딩 층을 주입할 수 있는 방식입니다. 이 방법은 무작위로 특정 차원으로 입력 벡터들을 뿌린 후 학습을 통해 가중치를 조정해 나가는 방식입니다.
 
 즉, 단어 사이의 관계를 반영하는 방식은 아닙니다.


In [2]:
from tensorflow.keras.models import  Sequential
from tensorflow.keras.layers import Embedding

vocab_size = 20000
max_len = 500

model = Sequential()
model.add(Embedding(vocab_size, 128, input_length= max_len))

# 2.word2vec

- word2vec의 핵심 아이디어는 "친구를 보면 그 사람을 알 수 있다" 입니다. 주변 단어와의 관계를 통해 해당 단어의 의미적 특성을 파악합니다.

- word2vec embedding matrix를 keras의 embedding에 주입하는 과정은 다음과 같습니다.

1. 구글에 사전 훈련된 word2vec bin 파일을 다운로드합니다.

2. gensim 모듈과 bin 파일을 활용해 word2vec 모델을 로드합니다.

In [7]:
!pip install -U gensim

     |████████████████████████████████| 23.9 MB 94 kB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [18]:
import gensim

import gensim.downloader as api
# wv = api.load("word2vec-google-news-300")
# word2vec = gensim.models.KeyedVectors.load_word2vec_format(wv, binary=True)

3. vocabulary에 있는 토큰들의 벡터를 가져와 embedding matrix에 저장합니다.

In [ ]:
embedding_matrix = np.zeros((vocab_size, 300)) # 300차원의 임베딩 매트릭스 생성

for index, word in enumerate(vocabulary)        #vocabulary에 있는 토큰들을 하나씩 넘겨줍니다.
  if word in word2vec:                          #넘겨 받은 토큰이 word2vec에 존재하면(이미 훈련이 된 토큰이라는 뜻)
    embedding_vector = word2vec[word]           #해당 토큰에 해당하는 vector를 불러오고
    embedding_matrix[index] = embedding_vector  #해당 위치의 embedding_matrix에 저장합니다.
  else:
    print("word2vec에 없는 단어 입니다.")
    break

4. keras embedding layer에 embedding_matrix를 가중치로 주어 이용합니다.

In [ ]:
model = Sequential()
model.add(Embeddding(vocab_size, 300, weights=[embedding_matrix], input_length = max_len))

# glove

- glove는 word2vec의 단점을 보완하기 위해 등장했습니다. word2vec이 사용자가 지정한 주변 단어의 개수에 대해서만 학습이 이루어지기 때문에 전체에 대한 정보를 담기 어렵다는 단점을 지적하였습니다. glove의 핵심 아이디어는 다음과 같습니다.

  각 토큰들 간의 유사성은 그대로 가져가면서 데이터 전체에 대한 빈도를 반영하자

- glove embedding matrix를 keras의 embedding matrix에 주입하는 방법은 다음과 같습니다.

1. 사전 훈련된 벡터를 갖고 있는 txt파일을 다운로드 합니다.

2. txt 파일에 있는 단어와 벡터들을 glove dictionary에 저장합니다.

In [ ]:
glove = dict()
f = open('./glove.txt')
for line in f:
  values = line.split()
  word  = values[0]
  vector = asarray(values[1:], dtype='float32')
  glove[word] =vector

f.close()

3. vocabulary에 있는 토큰들의 벡터를 가져와 embedding matrix에 저장합니다.

In [ ]:
embedding_matrix = np.zeros((vocab_size, 300))      #300차원의 임베딩 매트릭스 생성
for i, word in enumerate(vocabulary):               #vocabulary에 있는 토큰들을 하나씩 넘겨줍니다.
  if word in glove:                                 #넘겨 받은 토큰이 glove에 존재하면(이미훈련이 된 토큰 이라는뜻)
    embedding_vector = glove[word]                  #해당 토큰에 해당하는 vector를 불러오고
    embedding_matrix[i] = embedding_vector          #해당 위치의 embedding_matrix에 저장합니다.
  else : 
    print("glove에 없는 단어입니다.")
    break

4. keras embedding layer에 embedding_matrix를 가중치로 주어 이용합니다.

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_len))

#4. Fasttext

- Fasttext의 핵심 아이디어는 단어 단위가 아닌 sub 단어를 단위로 사용합니다. 즉 다음과 같습니다.

  word2vec -> 'apple'학습

  FastText -> "ap", "pp","pl", "le" 학습

- 따라서 미리 학습되지 않은 단어들에 대한 vector 표현도 해준다는 장점이있습니다.

1. 사전 훈련된 bin 파일을 다운로드 합니다.

2. vec 파일을 gensim을 활용하여 읽어옵니다.

In [ ]:
from gensim.models.keyedvectors import KeyedVectors
FastText = KeyedVectors.load_word2vec_format("./fasttext.bin",binary=True)

3. vocabulary에 있는 토큰들의 벡터를 가져와 embedding matrix에 저장합니다.

In [ ]:
embedding_matrix = np.zeros((vocab_size, 300))    #300차원의 임베딩 매트릭스 생성
for i, word in enumerate(vocabulary):
  if word in FastText:                            
    embedding_vector = FastText[word]
    embedding_matrix[i] = embedding_vector

4. keras embedding layer에 embedding_matrix를 가중치로 주어 이용합니다.

In [ ]:
model =Sequential()
model.add(Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_len))